In [1]:
import pandas as pd
pd.options.display.max_columns = 99
import chardet

In [2]:
with open('OneDrive\Documents\my_datasets\Crunchbase-investments.csv', mode='rb') as file:
    raw_bytes = file.read(40000)
    detected_encoding = chardet.detect(raw_bytes)
print(detected_encoding)

{'encoding': 'Windows-1252', 'confidence': 0.7295035139424166, 'language': ''}


ISO-8859-1 (also called Latin-1) is identical to Windows-1252 (also called CP1252) except for the code points 128-159 (0x80-0x9F).

In [3]:
Data = pd.read_csv('OneDrive\Documents\my_datasets\Crunchbase-investments.csv', encoding='ISO 8859-1', nrows=5)
Data

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000


In [4]:
chunk_iter = pd.read_csv('OneDrive\Documents\my_datasets\Crunchbase-investments.csv', 
                         encoding='ISO 8859-1', chunksize=5000)
rows = 0
for chunk in chunk_iter:
    rows += len(chunk)
print(rows)

52870


#### column's missing value counts

In [5]:
chunk_iter = pd.read_csv('OneDrive\Documents\my_datasets\Crunchbase-investments.csv', 
                         encoding='ISO 8859-1', chunksize=5000)
mv_list = []
for chunk in chunk_iter:
    mv_list.append(chunk.isnull().sum())
    
combined_mv_vc = pd.concat(mv_list)
unique_combined_mv_vc = combined_mv_vc.groupby(combined_mv_vc.index).sum()
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

#### Total memory footprint for each column


In [6]:
chunk_iter = pd.read_csv('OneDrive\Documents\my_datasets\Crunchbase-investments.csv', 
                         encoding='ISO 8859-1', chunksize=5000)
series_memory_fp = 0
for chunk in chunk_iter:
        series_memory_fp += chunk.memory_usage(deep=True)

# Drop memory footprint calculation for the index.
series_memory_fp = series_memory_fp.drop('Index')
series_memory_fp

company_permalink         4057788
company_name              3591326
company_category_code     3421104
company_country_code      3172176
company_state_code        3106051
company_region            3411545
company_city              3505886
investor_permalink        4980548
investor_name             3915666
investor_category_code     622424
investor_country_code     2647292
investor_state_code       2476607
investor_region           3396281
investor_city             2885083
funding_round_type        3410707
funded_at                 3542185
funded_month              3383584
funded_quarter            3383584
funded_year                422960
raised_amount_usd          422960
dtype: int64

#### Total memory footprint of the data (in megabytes)

In [7]:
series_memory_fp.sum() / (1024 * 1024)

56.98753070831299

In [8]:
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [9]:
# Drop columns representing URL's or containing way too many missing values (>90% missing)
drop_cols = ['investor_permalink', 'company_permalink', 'investor_category_code']
keep_cols = chunk.columns.drop(drop_cols)
keep_cols

Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')

In [10]:
chunk_iter = pd.read_csv('OneDrive\Documents\my_datasets\Crunchbase-investments.csv', 
                         encoding='ISO 8859-1', chunksize=5000)

col_types = {}
for chunk in chunk_iter:
    chunk['funded_month'] = chunk['funded_month'].str.replace(r'[0-9]{4}-','')
    chunk['funded_month'] = pd.to_numeric(chunk['funded_month'])
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))

In [11]:
uniq_col_types = {}
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
uniq_col_types

{'company_permalink': {'object'},
 'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_permalink': {'object'},
 'investor_name': {'object'},
 'investor_category_code': {'float64', 'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'funded_at': {'object'},
 'funded_month': {'float64', 'int64'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'raised_amount_usd': {'float64'}}

In [12]:
chunk

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,/company/nuorder,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,/person/mortimer-singer,Mortimer Singer,NaN,NaN,NaN,unknown,NaN,series-a,2012-10-01,10,2012-Q4,2012,3060000.0
50001,/company/chacha,ChaCha,advertising,USA,IN,Indianapolis,Carmel,/person/morton-meyerson,Morton Meyerson,NaN,NaN,NaN,unknown,NaN,series-b,2007-10-01,10,2007-Q4,2007,12000000.0
50002,/company/binfire,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,/person/moshe-ariel,Moshe Ariel,NaN,NaN,NaN,unknown,NaN,angel,2008-04-18,4,2008-Q2,2008,500000.0
50003,/company/binfire,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,/person/moshe-ariel,Moshe Ariel,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,1,2010-Q1,2010,750000.0
50004,/company/unified-color,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,/person/mr-andrew-oung,Mr. Andrew Oung,NaN,NaN,NaN,unknown,NaN,angel,2010-01-01,1,2010-Q1,2010,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52865,/company/garantia-data,Garantia Data,enterprise,USA,CA,SF Bay,Santa Clara,/person/zohar-gilon,Zohar Gilon,NaN,NaN,NaN,unknown,NaN,series-a,2012-08-08,8,2012-Q3,2012,3800000.0
52866,/company/duda-mobile,DudaMobile,mobile,USA,CA,SF Bay,Palo Alto,/person/zohar-gilon,Zohar Gilon,NaN,NaN,NaN,unknown,NaN,series-c+,2013-04-08,4,2013-Q2,2013,10300000.0
52867,/company/sitebrains,SiteBrains,software,USA,CA,SF Bay,San Francisco,/person/zohar-israel,zohar israel,NaN,NaN,NaN,unknown,NaN,angel,2010-08-01,8,2010-Q3,2010,350000.0
52868,/company/comprehend-systems,Comprehend Systems,enterprise,USA,CA,SF Bay,Palo Alto,/person/zorba-lieberman,Zorba Lieberman,NaN,NaN,NaN,unknown,NaN,series-a,2013-07-11,7,2013-Q3,2013,8400000.0


## Loading Chunks Into SQLite

In [13]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
chunk_iter = pd.read_csv('OneDrive\Documents\my_datasets\Crunchbase-investments.csv', 
                         encoding='ISO 8859-1', chunksize=5000)

col_types = {}
for chunk in chunk_iter:
    chunk['funded_month'] = chunk['funded_month'].str.replace(r'[0-9]{4}-','')
    chunk['funded_month'] = pd.to_numeric(chunk['funded_month'])
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))
    chunk.to_sql("investments", conn, if_exists='append', index=False)

In [14]:
df_info = pd.read_sql('PRAGMA table_info(investments)', conn)
print(df_info)

    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

In [15]:
len_crunchbase = pd.read_sql('SELECT count(*) AS Length from investments', conn)
print(len_crunchbase)

   Length
0  105740
